In [ ]:
import whisper
modelWhisper = whisper.load_model('medium')

Servidor Multihilo

In [ ]:
from flask import Flask, request, jsonify
from threading import Lock

import time
# Asegúrate de importar tu modelo Whisper correctamente
# from tu_paquete import modelWhisper

app = Flask(__name__)

# Crea un bloqueo para proteger el código contra la concurrencia
transcribe_lock = Lock()

@app.route('/transcribe', methods=['POST'])
def transcribe_audio():
    # Comprueba si el archivo fue enviado
    if 'file' not in request.files:
        return jsonify(error="No file part"), 400

    file = request.files['file']

    # Comprueba si el usuario no seleccionó un archivo
    if file.filename == '':
        return jsonify(error="No selected file"), 400

    # Genera un nombre de archivo único utilizando una marca de tiempo
    timestamp = int(time.time() * 1000)  # Marca de tiempo en milisegundos
    mp3_filepath = f"received_audio_{timestamp}.mp3"
    file.save(mp3_filepath)

    # Transcribe el archivo MP3 (Asegúrate de tener el modelo cargado correctamente)
    # Transcribe el archivo MP3 dentro de una sección crítica protegida por un bloqueo
    with transcribe_lock:    
        transcripcion = modelWhisper.transcribe(mp3_filepath, fp16=False)

    # Aquí estoy comentando la transcripción real y devolviendo una cadena de marcador de posición
    # Una vez que tengas tu modelo Whisper configurado correctamente, descomenta la línea de transcripción anterior
    # transcripcion = "Transcripción de ejemplo"

    # # Devuelve la transcripción
    return jsonify(transcripcion=transcripcion)
    # Devuelve la transcripción
    # return jsonify(transcripcion={"text": transcripcion})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5500, threaded=True)


Cliente

In [ ]:
import requests
import threading

import speech_recognition as sr
from pydub import AudioSegment


from pydub import AudioSegment, silence


from pydub import AudioSegment, silence

def quitar_silencios(input_filepath, output_filepath, min_silence_len=1000, new_silence_len=500, silence_thresh=-50):
    """
    Elimina silencios largos de un archivo de audio.

    Parámetros:
    - input_filepath: ruta al archivo de audio de entrada (MP3).
    - output_filepath: ruta al archivo de audio de salida (MP3).
    - min_silence_len: duración mínima del silencio a eliminar (en milisegundos).
    - new_silence_len: duración de los nuevos segmentos de silencio (en milisegundos).
    - silence_thresh: umbral de silencio (en dB).
    """
    
    # Cargar el archivo de audio
    audio_segment = AudioSegment.from_mp3(input_filepath)
    
    # Encuentra los segmentos de audio separados por silencios
    segments = silence.split_on_silence(audio_segment, min_silence_len=min_silence_len, silence_thresh=silence_thresh)

    # Crear un nuevo segmento de audio con silencios ajustados
    new_audio_segment = AudioSegment.empty()
    silence_chunk = AudioSegment.silent(duration=new_silence_len)  # Chunk de silencio de la nueva duración

    # Añade cada segmento de audio al nuevo audio, intercalando con los nuevos segmentos de silencio
    for segment in segments:
        new_audio_segment += segment + silence_chunk

    # Removemos el último chunk de silencio añadido
    new_audio_segment = new_audio_segment[:-new_silence_len]

    # Guarda el nuevo archivo de audio
    new_audio_segment.export(output_filepath, format="mp3")



from pydub import AudioSegment

def es_silencioso(filepath, silence_thresh=-55):
    """
    Detecta si un archivo de audio es completamente silencioso.

    Parámetros:
    - filepath: ruta al archivo de audio (MP3).
    - silence_thresh: umbral de silencio (en dB).

    Retorna:
    - True si el archivo es silencioso, False en caso contrario.
    """

    # Cargar el archivo de audio
    audio_segment = AudioSegment.from_mp3(filepath)
    
    # Obtener el nivel de dB del archivo
    db_level = audio_segment.dBFS

    # Comprobar si el nivel de dB está por debajo del umbral de silencio
    return db_level < silence_thresh



# Inicializa el reconocedor de voz
recognizer = sr.Recognizer()

iteration = 0

def send_audio_to_server(mp3_file, wav_file):
    # Convierte el archivo WAV a MP3
    AudioSegment.from_wav(wav_file).export(mp3_file, format="mp3")

    mp3_file_sin_silencios = f"{mp3_file}_sin_silencios.mp3"
    # quitar silencios
    quitar_silencios(mp3_file, mp3_file_sin_silencios)
    
    # Envía el archivo MP3 a un servidor
    url = "http://ia.javiergimenez.es:5500/transcribe"
    
    if not es_silencioso(mp3_file):
        with open(mp3_file_sin_silencios, 'rb') as f:
            response = requests.post(url, files={'file': f})

        # Muestra la respuesta del servidor
        response_data = response.json()  # Deserializa la respuesta JSON a un diccionario
        transcripcion = response_data.get("transcripcion", {}).get("text", "No se pudo obtener la transcripción")
        print("Respuesta del servidor:", transcripcion)

    else:
        print("Archivo silencioso, no se envía al servidor")

while True:
    # Captura el audio del micrófono
    with sr.Microphone() as source:      
        print("Capturando audio...")
        audio_data = recognizer.listen(source)
        print("Captura completa.")

    # Guarda el audio capturado como un archivo WAV
    wav_file = f"captured_audio_{iteration % 2}.wav"
    with open(wav_file, "wb") as f:
        f.write(audio_data.get_wav_data())

    # Define el nombre del archivo MP3
    mp3_file = f"captured_audio_{iteration % 2}.mp3"

    # Inicia un nuevo hilo para convertir y enviar el archivo MP3 al servidor
    threading.Thread(target=send_audio_to_server, args=(mp3_file, wav_file)).start()

    # Incrementa el contador de iteraciones
    iteration += 1


Servidor bloqueante (inicial que hice)

In [ ]:
from flask import Flask, request, jsonify
# Asegúrate de importar tu modelo Whisper correctamente
# from tu_paquete import modelWhisper

app = Flask(__name__)

@app.route('/transcribe', methods=['POST'])
def transcribe_audio():
    # Comprueba si el archivo fue enviado
    if 'file' not in request.files:
        return jsonify(error="No file part"), 400

    file = request.files['file']

    # Comprueba si el usuario no seleccionó un archivo
    if file.filename == '':
        return jsonify(error="No selected file"), 400

    # Guarda el archivo en el servidor
    mp3_filepath = "received_audio.mp3"
    file.save(mp3_filepath)

    # Transcribe el archivo MP3 (Asegúrate de tener el modelo cargado correctamente)
    transcripcion = modelWhisper.transcribe(mp3_filepath)

    # Aquí estoy comentando la transcripción real y devolviendo una cadena de marcador de posición
    # Una vez que tengas tu modelo Whisper configurado correctamente, descomenta la línea de transcripción anterior
    # transcripcion = "Transcripción de ejemplo"

    # Devuelve la transcripción
    return jsonify(transcripcion=transcripcion)

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5500)
